# Ice-thickness length scale smoothing

We are implmenting eq 2 and 3 from https://polarresearch.net/index.php/polar/article/view/3498/9172

In [ ]:
from scipy.ndimage import gaussian_filter, generic_filter
import xarray as xr

In [ ]:
from pathlib import Path
from typing import Dict, List, Optional, Union

import cartopy.crs as ccrs
import cf_xarray.units  # pylint: disable=unused-import
import geopandas as gp
import matplotlib
import numpy as np
import pint_xarray  # pylint: disable=unused-import
import pylab as plt
import seaborn as sns
import xarray as xr
from matplotlib import cm, colors
from matplotlib.colors import LightSource
from shapely import get_coordinates

from glacier_flow_tools.utils import (
    blend_multiply,
    figure_extent,
    get_dataarray_extent,
    register_colormaps,
)

register_colormaps()


## This plotting function makes a nice hillshade so it's easier to validate the results

In [ ]:
def plot_glacier(
    surface: xr.DataArray,
    overlay: xr.DataArray,
    cmap: str = "viridis",
    vmin: float = 10,
    vmax: float = 1500,
    ticks: Union[List[float], np.ndarray] = [10, 100, 250, 500, 750, 1500],
    fontsize: float = 6,
    figwidth: float = 3.2,
):
    """
    Plot a surface over a hillshade, add profile and correlation coefficient.

    This function plots a surface over a hillshade, adds a profile and correlation coefficient.
    The plot is saved as a PDF file in the specified result directory.

    Parameters
    ----------
    surface : xr.DataArray
        The surface to be plotted over the hillshade.
    overlay : xr.DataArray
        The overlay to be added to the plot.
    result_dir : Union[str, Path]
        The directory where the result PDF file will be saved.
    cmap : str, optional
        The colormap to be used for the plot, by default "viridis".
    vmin : float, optional
        The minimum value for the colormap, by default 10.
    vmax : float, optional
        The maximum value for the colormap, by default 1500.
    ticks : Union[List[float], np.ndarray], optional
        The ticks to be used for the colorbar, by default [10, 100, 250, 500, 750, 1500].
    fontsize : float, optional
        The font size to be used for the plot, by default 6.
    figwidth : float, optional
        The width of the figure in inches, by default 3.2.

    Examples
    --------
    >>> plot_glacier(profile_series, surface, overlay, '/path/to/result_dir')
    """
    plt.rcParams["font.size"] = fontsize
    cartopy_crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)
    # Shade from the northwest, with the sun 45 degrees from horizontal
    light_source = LightSource(azdeg=315, altdeg=45)
    glacier_overlay = overlay
    glacier_surface = surface.interp_like(glacier_overlay)

    extent = get_dataarray_extent(glacier_overlay)
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    mapper = cm.ScalarMappable(norm=norm, cmap=cmap)

    v = mapper.to_rgba(glacier_overlay.to_numpy())
    z = glacier_surface.to_numpy()

    ar = 1.0  # initial aspect ratio for first trial
    wi = figwidth  # width in inches
    hi = wi * ar  # height in inches

    fig = plt.figure(figsize=(wi, hi))
    ax = fig.add_subplot(111, projection=cartopy_crs)
    rgb = light_source.shade_rgb(v, elevation=z, vert_exag=0.01, blend_mode=blend_multiply)
    # Use a proxy artist for the colorbar...
    im = ax.imshow(v, cmap=cmap, vmin=vmin, vmax=vmax)
    im.remove()
    ax.imshow(rgb, extent=extent, origin="upper", transform=cartopy_crs)
    ax.gridlines(
        draw_labels={"top": "x", "left": "y"},
        dms=True,
        xlocs=np.arange(-50, 0, 1),
        ylocs=np.arange(50, 88, 1),
        x_inline=False,
        y_inline=False,
        rotate_labels=20,
        ls="dotted",
        color="k",
    )

    fig.colorbar(im, ax=ax, shrink=0.5, pad=0.025, label=overlay.units, extend="both", ticks=ticks)
    plt.draw()

    # Get proper ratio here
    xmin, xmax = ax.get_xbound()
    ymin, ymax = ax.get_ybound()
    y2x_ratio = (ymax - ymin) / (xmax - xmin)
    fig.set_figheight(wi * y2x_ratio)
    fig.tight_layout()
    plt.close()
    return fig


## We use Bedmachine as it has both surface and ice thickness

In [ ]:
ds = xr.open_dataset("/mnt/storstrommen/data/MCdataset/BedMachineGreenland-v5.nc")
# ds = xr.open_dataset("/Users/andy/Google Drive/My Drive/data/MCdataset/BedMachineGreenland-v5.nc")

## Select Jakobshavn

In [ ]:
jak_ds = ds.sel(x=slice(-226_000, -140_000), y=slice(-2_250_000, -2_300_000))

In [ ]:
dem = jak_ds["surface"]
plot_glacier(dem, dem, cmap="Grays", figwidth=16)

In [ ]:
dem_smoothed_gaussian = jak_ds["surface"].copy()
dem_smoothed_gaussian.values = gaussian_filter(jak_ds["surface"], 2)
plot_glacier(dem_smoothed_gaussian, dem_smoothed_gaussian, cmap="Grays", figwidth=16)

In [ ]:
H = jak_ds["thickness"].copy()

## Brinkerhoff, Aschwanden and Fahnestock (2021): DOI:10.1017/jog.2020.112

I think Eq 45 should give us a distance scaled sigma?

In [ ]:
l = 4  # length scale
L = l * (H.to_numpy() @ H.to_numpy().T)**(1/2) # I am not sure this is correct

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
X, Y = np.meshgrid(H.x, H.y)

In [ ]:
D = squareform(pdist(Y))

In [ ]:
Sigma = (1 + D**2 / (2 * L**2))**(-1)

## Gaussian smoothing

$$ \omega=\frac{1}{2 \pi \sigma^2} e^{-\frac{x^2+y^2}{2 \sigma^2}}$$

How do we implment this as a **generic_filter**? The implementation of **gaussian_filter** is here: https://github.com/scipy/scipy/blob/v1.13.1/scipy/ndimage/_filters.py#L286-L390.
It uses *np.correlate* instead of convolve, and loops over all axis. Can we do that too?

Maybe we can use https://github.com/scipy/scipy/blob/v1.13.1/scipy/ndimage/_filters.py#L186 for inspiration. We need to compute $\sigma$ scaled by ice thickness in a sensible way. 

In [ ]:
def gaussian_weighted_filter(dem, l, H, radius):
    x = np.arange(-radius, radius+1)
    L = l * (H @ H.T)**(1/2) # I am not sure this is correct
    D = squareform(pdist(np.vstack([x.ravel(), x.ravel()]).T))
    sigma = (1 + D**2 / (2 * L**2))**(-1)
    phi = 1 / (2 * np.pi * sigma**2) * np.exp(-(x**2 + y**2) / (2 * sigma**2) )
    return np.correlate(dem, phi)

In [ ]:
dem = jak_ds["surface"].to_numpy()
H = jak_ds["thickness"].to_numpy()
x = jak_ds.x.to_numpy()
y = jak_ds.y.to_numpy()
X, Y = np.meshgrid(x, y)
l = 4
m, n = dem.shape

In [ ]:
    L = l * (H @ H.T)**(1/2) # I am not sure this is correct
    X_dist = np.zeros((m*n, 2))
    X_dist[:, 0] = X.ravel()
    X_dist[:, 1] = Y.ravel()
    D = squareform(pdist(X_dist))

In [ ]:
# Co-Pilot 
import numpy as np
from scipy.ndimage import generic_filter

def gaussian(x, sigma):
    """
    Gaussian function.

    Parameters
    ----------
    x : float
        The distance from the center of the Gaussian.
    sigma : float
        The standard deviation of the Gaussian.

    Returns
    -------
    float
        The value of the Gaussian function at x.
    """
    return (1 / (2 * np.pi * sigma**2)) * np.exp(-x**2 / (2 * sigma**2))

def apply_gaussian_filter(input_array, sigma):
    """
    Apply a Gaussian filter to an array.

    Parameters
    ----------
    input_array : numpy.ndarray
        The input array to filter.
    sigma : float
        The standard deviation of the Gaussian filter.

    Returns
    -------
    numpy.ndarray
        The filtered array.
    """
    # Define the footprint of the filter to be the same size as the input array
    footprint = np.ones_like(input_array)

    # Define the function to apply to each neighborhood
    def func(neighborhood):
        # Calculate the distances from the center
        distances = np.abs(neighborhood - neighborhood[len(neighborhood) // 2])
        # Apply the Gaussian function to each distance
        return gaussian(distances, sigma)

    # Apply the filter
    return generic_filter(input_array, func, footprint=footprint)

In [ ]:
# Too much memory
apply_gaussian_filter(dem, 2)

In [ ]:
def _gaussian_weighted_kernel1d(l, H, x):
    """
    Computes a 1-D ice-thickness weighted Gaussian convolution kernel.
    """

    L = l * (H @ H.T)**(1/2) # I am not sure this is correct
    sigma = (1 + D**2 / (2 * L**2))**(-1)
    sigma2 = sigma * sigma
    phi_x = numpy.exp(-0.5 / sigma2 * x ** 2)
    phi_x = phi_x / phi_x.sum()

    return phi_x


In [ ]:
def gaussian_weighted_filter1d(input, l, H, x, axis=-1, order=0, output=None,
                      mode="reflect", cval=0.0):
    """1-D Gaussian filter.

    Parameters
    ----------
    %(input)s
    l : number of ice thicknesses
        ice thickness multiplier
    %(axis)s
    order : int, optional
        An order of 0 corresponds to convolution with a Gaussian
        kernel. A positive order corresponds to convolution with
        that derivative of a Gaussian.
    %(output)s
    %(mode_reflect)s
    %(cval)s

    Returns
    -------
    gaussian_weighted_filter1d : ndarray


    """
    sd = float(sigma)
    # make the radius of the filter equal to truncate standard deviations
    lw = int(truncate * sd + 0.5)
    if radius is not None:
        lw = radius
    if not isinstance(lw, numbers.Integral) or lw < 0:
        raise ValueError('Radius must be a nonnegative integer.')
    # Since we are calling correlate, not convolve, revert the kernel
    weights = _gaussian_weighted_kernel1d(l, H, x)[::-1]
    return correlate1d(input, weights, axis, output, mode, cval, 0)


In [ ]:
def gaussian_weighted_filter(input, l, H, x, output=None,
                    mode="reflect", cval=0.0,
                    axes=None):
    """Multidimensional Gaussian filter.

    Parameters
    ----------
    %(input)s
    sigma : scalar or sequence of scalars
        Standard deviation for Gaussian kernel. The standard
        deviations of the Gaussian filter are given for each axis as a
        sequence, or as a single number, in which case it is equal for
        all axes.
    order : int or sequence of ints, optional
        The order of the filter along each axis is given as a sequence
        of integers, or as a single number. An order of 0 corresponds
        to convolution with a Gaussian kernel. A positive order
        corresponds to convolution with that derivative of a Gaussian.
    %(output)s
    %(mode_multiple)s
    %(cval)s
    truncate : float, optional
        Truncate the filter at this many standard deviations.
        Default is 4.0.
    radius : None or int or sequence of ints, optional
        Radius of the Gaussian kernel. The radius are given for each axis
        as a sequence, or as a single number, in which case it is equal
        for all axes. If specified, the size of the kernel along each axis
        will be ``2*radius + 1``, and `truncate` is ignored.
        Default is None.
    axes : tuple of int or None, optional
        If None, `input` is filtered along all axes. Otherwise,
        `input` is filtered along the specified axes. When `axes` is
        specified, any tuples used for `sigma`, `order`, `mode` and/or `radius`
        must match the length of `axes`. The ith entry in any of these tuples
        corresponds to the ith entry in `axes`.

    Returns
    -------
    gaussian_filter : ndarray
        Returned array of same shape as `input`.

    Notes
    -----
    The multidimensional filter is implemented as a sequence of
    1-D convolution filters. The intermediate arrays are
    stored in the same data type as the output. Therefore, for output
    types with a limited precision, the results may be imprecise
    because intermediate results may be stored with insufficient
    precision.

    The Gaussian kernel will have size ``2*radius + 1`` along each axis. If
    `radius` is None, the default ``radius = round(truncate * sigma)`` will be
    used.

    Examples
    --------
    >>> from scipy.ndimage import gaussian_filter
    >>> import numpy as np
    >>> a = np.arange(50, step=2).reshape((5,5))
    >>> a
    array([[ 0,  2,  4,  6,  8],
           [10, 12, 14, 16, 18],
           [20, 22, 24, 26, 28],
           [30, 32, 34, 36, 38],
           [40, 42, 44, 46, 48]])
    >>> gaussian_filter(a, sigma=1)
    array([[ 4,  6,  8,  9, 11],
           [10, 12, 14, 15, 17],
           [20, 22, 24, 25, 27],
           [29, 31, 33, 34, 36],
           [35, 37, 39, 40, 42]])

    >>> from scipy import datasets
    >>> import matplotlib.pyplot as plt
    >>> fig = plt.figure()
    >>> plt.gray()  # show the filtered result in grayscale
    >>> ax1 = fig.add_subplot(121)  # left side
    >>> ax2 = fig.add_subplot(122)  # right side
    >>> ascent = datasets.ascent()
    >>> result = gaussian_filter(ascent, sigma=5)
    >>> ax1.imshow(ascent)
    >>> ax2.imshow(result)
    >>> plt.show()
    """
    input = numpy.asarray(input)
    output = _ni_support._get_output(output, input)

    axes = _ni_support._check_axes(axes, input.ndim)
    num_axes = len(axes)
    orders = _ni_support._normalize_sequence(order, num_axes)
    sigmas = _ni_support._normalize_sequence(sigma, num_axes)
    modes = _ni_support._normalize_sequence(mode, num_axes)
    radiuses = _ni_support._normalize_sequence(radius, num_axes)
    axes = [(axes[ii], sigmas[ii], orders[ii], modes[ii], radiuses[ii])
            for ii in range(num_axes) if sigmas[ii] > 1e-15]
    if len(axes) > 0:
        for axis, sigma, order, mode, radius in axes:
            gaussian_weighted_filter1d(input, l, H, x, axis, output,
                              mode, cval)
            input = output
    else:
        output[...] = input[...]
    return output

## Triangular smoothing

$$ \omega = \max\left( \frac{\sqrt{x^2+y^2}}{\sigma},0 \right)$$